In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.metrics import precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

train=pd.read_csv('../input/Kannada-MNIST/train.csv')
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')

# Visualising data

In [ ]:
img = train.drop('label',axis=1)

img = img.to_numpy()
one = img[0].reshape(28,28)
two = img[1].reshape(28,28)
three = img[2].reshape(28,28)
four = img[3].reshape(28,28)

In [ ]:
plt.imshow(one)

# Normalization

In [ ]:
training_labels=train.label

training_images=train.drop('label',axis=1)
  
# Normalizing and reshaping data
training_images=training_images.values.reshape(-1, 28, 28, 1)
training_images=training_images / 255.0

In [ ]:
training_labels = training_labels.to_numpy()
training_labels = training_labels.reshape(-1,1)

# Encoding

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
training_labels = enc.fit_transform(training_labels)

In [ ]:
training_labels = pd.DataFrame(training_labels)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(training_images, training_labels, test_size=0.2, random_state=20)

# Creating model

In [ ]:
def train_mnist_conv():  
    # Creating model
    model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3,3), input_shape=(28, 28, 1)),
            tf.keras.layers.PReLU(alpha_initializer='zeros'),
            tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
#             tf.keras.layers.Conv2D(32, (3,3)),
#             tf.keras.layers.PReLU(alpha_initializer='zeros'),
#             tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model
    
new_model = train_mnist_conv()
new_model.summary()

In [ ]:
history = new_model.fit(X_train, Y_train, epochs = 10)

# Accuracy and loss 

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Traaining accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc=0)
plt.figure()
 
plt.show()

In [ ]:
loss_train = history.history['loss']

plt.plot(loss_train, 'g', label='Training loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluating model

In [ ]:
X_test = X_test.reshape(-1, 28, 28, 1)
X_test = X_test/255.0

In [ ]:
preds = new_model.predict(X_test)

In [ ]:
new_model.evaluate(X_test, Y_test)

In [ ]:
Y_test = Y_test.to_numpy()

In [ ]:
new_preds = np.argmax(preds, axis=1)

In [ ]:
new_truth = np.argmax(Y_test, axis=1)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(new_truth, new_preds)
plt.figure(figsize=(8,8))
sns.heatmap(cm, fmt='.0f', annot=True, linewidths=0.2, linecolor='purple')
plt.xlabel('predicted value')
plt.ylabel('Truth value')
plt.show()

In [ ]:
test_id=test.id

test_images=test.drop('id',axis=1)

test_val=test.id

test_images = test_images.values.reshape(-1, 28, 28, 1)
test_images=test_images/255.0

y_pre=new_model.predict(test_images)     #making prediction
y_pre=np.argmax(y_pre,axis=1) #changing the prediction intro labels

sample_sub['label']=y_pre
sample_sub.to_csv('submission.csv',index=False)

sample_sub.head()